In [1]:
import os
%pwd

'/Users/songfxu/Documents/study/MLOps-End2End/research'

In [2]:
os.chdir("../")
%pwd

'/Users/songfxu/Documents/study/MLOps-End2End'

In [4]:
from pydantic import BaseModel
from pathlib import Path

class ModelTrainerConfig(BaseModel):
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

class ModelTrainerMgr:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name=config.model_name,
            alpha= params.alpha,
            l1_ratio= params.l1_ratio,
            target_column = schema.name
        )
        return model_trainer_config

In [7]:
import pandas as pd
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib

In [8]:
class ModelTrainer:
    def __init__(self,config = ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        lr = ElasticNet(l1_ratio=self.config.l1_ratio, alpha=self.config.alpha, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [9]:
try:
    config = ModelTrainerMgr().get_model_trainer_config()
    ModelTrainer(config=config).train()
except Exception as e:
    raise e

[2025-11-17 10:53:41,804: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-11-17 10:53:41,805: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-17 10:53:41,807: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-17 10:53:41,808: INFO: common: Created directory at: artifacts]
[2025-11-17 10:53:41,809: INFO: common: Created directory at: artifacts/model_trainer]
